In [14]:
import os, re, pickle
import pandas as pd
import numpy as np
from os.path import join
from tqdm import tqdm
# default style 경고문구 제거
import warnings
warnings.simplefilter("ignore")

# dtype 정의
restoration_dtype = {0:str, 1:str,6:str,19:str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
cost_dtype = {'채무자키':str, '계좌키':str, '법비용키':str, '관할법원코드':str}

# 원상회복 정의
# 원상회복원금 = 조정전원금 - 상환원금
# 원상회복비용= 조정전비용 - 상환비용
# 원상회복이자 = 조정전이자(신고미수이자+신고연체이자) - 납입이자 + 상환기간이자

In [2]:
desktop = r'C:\Users\SL\Desktop'
################################################################
wd = r"D:\3.자산\신용회복\솔림\1.신용회복\2.원상회복내역\분기별"
업무대장 = pd.read_excel(join(desktop, '0.new_신용회복 업무대장.xlsx'), sheet_name='data', usecols=['채무자키', '계좌키', '주민+계좌', '채무구분'], dtype='str')
################################################################
file_list = [x for x in os.listdir(wd) if not re.match('~', x)]

cols = ['No', '실효일', '진행상태', '고객명', '고객식별번호', '채무구분', '계좌번호', '재조정여부', '수정조정여부', 
    '조정전원금', '조정전비용', '조정전이자', '조정후원금', '조정후기타채무', 
    '납입원금', '납입비용', '미감면이자', '납입이자', '상환기간이자', '연체이자기산일', 
    '원상회복원금', '원상회복비용', '원상회복이자']

In [3]:
restoration_ori = pd.DataFrame(None)
for f in file_list : 
    temp = pd.read_excel(join(wd, f), skiprows=4, header=None, dtype = restoration_dtype)
    temp.dropna(subset=[0], inplace=True) # 0칼럼이 빈값인 경우 삭제.(설명행)
    restoration_ori = pd.concat([restoration_ori, temp], axis=0)

# 칼럼명 변경
restoration_ori.columns = cols

In [18]:
# 편집본 카피
restoration = restoration_ori.copy()

In [19]:
# 보증채무 삭제
restoration = restoration.query('채무구분=="주채무"')

# 계좌번호 문제 있는 거 변경
restoration.계좌번호.str.replace('BNK216LO071756-0002', '216LO071756-0002')
restoration.계좌번호.str.replace('YM-13-1-12279-02', 'TYM-13-1-12279-02')

# 주민+계좌 열 추가
restoration["주민+계좌"] = restoration.고객식별번호 + restoration.계좌번호

# 원상회복OPB열 추가
restoration["원상회복OPB"] = restoration.원상회복원금 + restoration.원상회복비용
restoration["원상회복OPB이자"] = restoration.원상회복OPB + restoration.원상회복이자

# 실효일(취소) 형식 지정
restoration["실효일"] = restoration["실효일"].str[:4] + "-" + restoration["실효일"].str[4:6] +"-"+restoration["실효일"].str[6:]

# 업무대장과 병합하여 키값 연결
restoration = restoration.merge(업무대장, on=['주민+계좌','채무구분'], how='left')

In [6]:
# # 계좌키 파일저장 > 엑셀 입금 조회 및 계좌 조회
# restoration.계좌키.to_excel(join(desktop, "실효자계좌키.xlsx"), index=False)

In [7]:
# 입금조회새창, 계좌조회새창, 법비용새창 읽기
deposit = pd.read_excel(join(desktop, "입금조회새창_20231102_1111.xlsx"), dtype=deposit_dtype)
account = pd.read_excel(join(desktop, "계좌조회새창_20231102_1140.xlsx"), dtype=account_dtype)[["채무자키", "계좌키", "현재원금", "현재법비용", "현재미수이자", "현재연체이자"]]
cost = pd.read_excel(join(desktop, "법비용새창_20231102_1530.xlsx"), dtype=cost_dtype)[["채무자키","계좌키", "법비용일", "법비용합계"]]

# deposit 조작
deposit.sort_values('회수합계', ascending=False) # 최신순(회수합계 많은 순)으로 정렬 (날짜는 중복될 수 있음)
deposit = deposit.drop(deposit[deposit.입금기타>0].index, axis=0).reset_index(drop=True) # 부채증명서 행삭제

In [20]:
# 실효일 당시의 현재금액 구하기
현재원금들, 현재비용들, 현재미수이자들 = [],[],[]
for i, v in tqdm(restoration.iterrows(), total = len(restoration)) :
    
    dt = deposit.query('계좌키 == @v.계좌키') # len(dt) 특정불가
    if len(dt) > 0 : # 입금기록이 있는 경우
        date_list = dt.입금일.to_list()
        date_list.append(v.실효일) # 실효일 추가
        date_list.sort()
        date_list.reverse() # 역정렬

        실효일index = date_list.index(v.실효일)
        if 실효일index >  0 : # 실효일 다음 입금액이 있는 경우 : 입금새창에서 현재금액 구한다.
            계좌사용 = False
        else : # 실효일 이후 입금이 없음 > 계좌의현재금을 끌어오거나 최근 입금의 현재금액 - 입금금액 
            계좌사용 = True
    else : # 입금기록이 없는 경우
        계좌사용 = True

    if 계좌사용 :
        at = account.query('계좌키 == @v.계좌키') # len(at) == 1 이므로 무조건 values[0]
        ct = cost.query('계좌키 == @v.계좌키 and 법비용일>@v.실효일') # 법비용
        # 실효일 이후 법비용합계
        실효후비용 = ct.법비용합계.sum()

        현재원금들.append(at.현재원금.values[0])
        현재비용들.append(at.현재법비용.values[0]-실효후비용)
        현재미수이자들.append(at.현재미수이자.values[0])
    else : # 실효일 바로 다음(-1)의 현재금액 그대로
        현재원금들.append(dt.현재원금.values[실효일index-1])
        현재비용들.append(dt.현재법비용.values[실효일index-1])
        현재미수이자들.append(dt.현재미수이자.values[실효일index-1])  

# 원상회복에 현재금액들 추가하기
restoration["실효당시현재원금"] = 현재원금들
restoration["실효당시현재비용"] = 현재비용들
restoration["실효당시현재미수이자"] = 현재미수이자들
restoration["실효당시OPB"] = restoration["실효당시현재원금"] + restoration["실효당시현재비용"]
restoration["실효당시OPB이자"] = restoration["실효당시OPB"] + restoration["실효당시현재미수이자"]
restoration["원금비교"] = restoration.원상회복원금 - restoration.실효당시현재원금
restoration["비용비교"] = restoration.원상회복비용 - restoration.실효당시현재비용
restoration["미수이자비교"] = restoration.원상회복이자 - restoration.실효당시현재미수이자
restoration["OPB비교"] = restoration.원상회복OPB - restoration.실효당시OPB
restoration["합계비교"] = restoration.원상회복OPB이자 - restoration.실효당시OPB이자

100%|██████████| 666/666 [00:02<00:00, 310.09it/s]


In [21]:
# 컬럼 편집
restoration["No"] = range(1,len(restoration)+1)
restoration.insert(1, "계좌키", restoration.pop("계좌키"))
restoration.insert(1, "채무자키", restoration.pop("채무자키"))
restoration.drop(columns=["주민+계좌"], inplace=True)

In [22]:
# 최종본 엑셀 저장
restoration.to_excel(join(desktop, "2023신복실효건 원상회복금액 vs 전산금액.xlsx"), index=False)